In [1]:
import pandas as pd

data = pd.read_csv('data.csv')
data.head()

,ID,air_time1,disp_index1,gmrt_in_air1,gmrt_on_paper1,max_x_extension1,max_y_extension1,mean_acc_in_air1,mean_acc_on_paper1,mean_gmrt1,...,mean_jerk_in_air25,mean_jerk_on_paper25,mean_speed_in_air25,mean_speed_on_paper25,num_of_pendown25,paper_time25,pressure_mean25,pressure_var25,total_time25,class
0,id_1,5160,0.000013,120.804174,86.853334,957,6601,0.361800,0.217459,103.828754,...,0.141434,0.024471,5.596487,3.184589,71,40120,1749.278166,296102.7676,144605,P
1,id_2,51980,0.000016,115.318238,83.448681,1694,6998,0.272513,0.144880,99.383459,...,0.049663,0.018368,1.665973,0.950249,129,126700,1504.768272,278744.2850,298640,P
2,id_3,2600,0.000010,229.933997,172.761858,2333,5802,0.387020,0.181342,201.347928,...,0.178194,0.017174,4.000781,2.392521,74,45480,1431.443492,144411.7055,79025,P
3,id_4,2130,0.000010,369.403342,183.193104,1756,8159,0.556879,0.164502,276.298223,...,0.113905,0.019860,4.206746,1.613522,123,67945,1465.843329,230184.7154,181220,P
4,id_5,2310,0.000007,257.997131,111.275889,987,4732,0.266077,0.145104,184.636510,...,0.121782,0.020872,3.319036,1.680629,92,37285,1841.702561,158290.0255,72575,P


In [2]:
data['class']= data['class'].map({'P':1, 'H':0})
data['class'].head()

0    1
1    1
2    1
3    1
4    1
Name: class, dtype: int64

In [3]:
X = data.drop(columns=['ID', 'class'], axis=1).to_numpy()

In [4]:
y = data['class'].to_numpy()

In [5]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1, random_state=42)
X_train.shape

(156, 450)

In [7]:
from keras.layers import Input, Dense, Dropout, Conv1D, Flatten,MaxPooling1D, Conv2D
from keras.regularizers import l1_l2
from keras.models import Sequential
from keras.optimizers import Adam
import keras
import tensorflow as tf

2024-07-21 08:09:09.567586: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 08:09:09.664072: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-21 08:09:09.739077: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-21 08:09:09.785826: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-21 08:09:09.800196: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-21 08:09:09.900883: I tensorflow/core/platform/cpu_feature_gu

In [13]:
# Define model parameters
input_shape = (450, 1) 
num_classes = 2

In [17]:
# Create a sequential model
model = Sequential()

# Add convolutional layer with zero-padding
model.add(Conv1D(8,kernel_size=3,strides=1,padding="same",input_shape=(450,1),activation="relu"))

In [18]:
# Add four fully connected layers with ReLU activation
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dense(64, activation="relu"))

In [23]:
# Add output dense layer with softmax activation
model.add(Dense(1, activation="softmax"))

# Compile the model with Adam optimizer and categorical cross-entropy loss
model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=["accuracy"])

# Print a summary of the model layers
model.summary()



Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 450, 8)         │            32 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 3600)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 512)            │     1,843,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 2)              │           130 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │             3 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 1)              │             2 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,016,359 (7.69 MB)

 Trainable params: 2,016,359 (7.69 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
model.fit(X_train, y_train, epochs=10, batch_size=50, validation_data=(X_test, y_test), validation_batch_size=5)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 78ms/step - accuracy: 0.5191 - loss: 0.6925 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5431 - loss: 0.6913 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5085 - loss: 0.6930 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5298 - loss: 0.6920 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.5091 - loss: 0.6930 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.5338 - loss: 0.6919 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.5138 - loss: 0.6928 - val_accuracy: 0.5000 - val_loss: 0.6935
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.5145 - loss: 0.6927 - val_accuracy: 0.5000 - val_loss: 0.6935
